## 🥈 Silver Layer

### 📦 Import the Necessary Packages 

In [1]:
import pandas as pd
import numpy as np
import string
import datetime
import re

### 🔃 Load the Bronze Layer Data 

In [2]:
df = pd.read_parquet('./Medallion Architecture/Bronze/Bronze.parquet')
df.head()

,Student Name,House,Year,Wand Type,Pet,Potions Grade,Defense Against the Dark Arts Grade,Transfiguration Grade,Spells Learned,Quidditch Position,Points Earned for House,Detentions,Participation in Dueling Club,Triwizard Tournament Involvement,Items Owned,Knuts Spent in Hogsmeade,Attendance at Classes,Magical Accidents,fileName,loadDateTimeStamp
0,Seamus Potter,Slytherin,2,"9 inches, elm, veela hair core",Dragon,Outstanding,Exceeds Expectations,Outstanding,26,Beater,-100,3,No,No,"Time-Turner, Marauder's Map",88,74,5,Hogwarts_Student_Data_20240101.csv,2024-12-05 22:53:29.684926
1,Ginny Spinnet,Gryffindor,6,"9 inches, elm, veela hair core",Cat,Exceeds Expectations,Acceptable,Outstanding,11,Chaser,139,2,No,No,"Firebolt, Sneakoscope, Extendable Ears",99,67,6,Hogwarts_Student_Data_20240101.csv,2024-12-05 22:53:29.684926
2,Padma Parkinson,Gryffindor,4,"11 inches, holly, phoenix feather core",Muggle Born,Outstanding,Acceptable,Outstanding,19,Seeker,93,2,No,No,Invisibility Cloak,342,88,6,Hogwarts_Student_Data_20240101.csv,2024-12-05 22:53:29.684926
3,None,Ravenclaw,1,"12 inches, yew, unicorn hair core",Toad,Exceeds Expectations,Exceeds Expectations,Exceeds Expectations,16,Seeker,-73,2,No,None,Time-Turner,233,45,5,Hogwarts_Student_Data_20240101.csv,2024-12-05 22:53:29.684926
4,None,Ravenclaw,1,"10 inches, oak, dragon heartstring core",Owl,None,Outstanding,Exceeds Expectations,9,Keeper,90,0,No,Yes,"Marauder's Map, Sneakoscope",473,27,7,Hogwarts_Student_Data_20240101.csv,2024-12-05 22:53:29.684926


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48000 entries, 0 to 47999
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Student Name                         34560 non-null  object        
 1   House                                44000 non-null  object        
 2   Year                                 48000 non-null  int64         
 3   Wand Type                            48000 non-null  object        
 4   Pet                                  40054 non-null  object        
 5   Potions Grade                        44560 non-null  object        
 6   Defense Against the Dark Arts Grade  48000 non-null  object        
 7   Transfiguration Grade                48000 non-null  object        
 8   Spells Learned                       48000 non-null  int64         
 9   Quidditch Position                   38496 non-null  object        
 10  Points Ear

### 🧼 Data Validation and Cleaning Functions

### ✅ Check for Null Values

### 1. 🏐 Replace Nulls with the Most Frequent Column Value

In [4]:
def fill_nulls_with_most_frequent(df) -> pd.DataFrame:
    for col in df.columns:
        if df[col].isnull().any():
            most_frequent_value = df[col].value_counts().index[0]
            df[col] = df[col].fillna(most_frequent_value)
    return df


### 2.  ✨ Clean Punctuation Errors in `Quidditch Position`

In [5]:
def clean_quidditch_position(df):
    df['Quidditch Position'] = df['Quidditch Position'].str.replace(f"[{string.punctuation}\\\\]", "", regex=True)
    return df


### 3. 🕵️‍♂️ Check for Misspellings in `Student Name`

In [6]:
def check_similarity(word1: str, word2: str) -> float:
    word_set1 = set(word1)
    word_set2 = set(word2)

    intersection = word_set1.intersection(word_set2)
    intersection_length = len(intersection)

    total_char_count = len(word_set1.union(word_set2))
    similarity =intersection_length / total_char_count
    return similarity

In [7]:
def check_misspelling(df: pd.DataFrame, column: str, threshold: float) -> pd.DataFrame:
    all_unique_values = list(set(df[column].tolist()))

    similarity_list = []

    for i in range(len(all_unique_values)):
        value1 = all_unique_values[i]
        for j in range(i + 1, len(all_unique_values)):
            value2 = all_unique_values[j]
            similarity = round(check_similarity(value1, value2),4)
            if similarity > threshold:
                similarity_list.append((value1, value2, similarity))
    return pd.DataFrame(similarity_list, columns=['Value 1', 'Value 2', 'Similarity'])

### 4. 🏠 Clean and Capitalize `House` Names

In [8]:
def clean_House(df):
    df['House'] = df['House'].str.lower()
    df['House'] = df['House'].apply(lambda x: x.capitalize())
    return df


### 5. ✍️ Correct Misspelled `Harry Potter` in `Student Name`

In [9]:
def generate_corrections_mapping(misspellings_df, correct_name):
    harry_potter_similarities = misspellings_df[
        (misspellings_df['Value 1'] == correct_name) | (misspellings_df['Value 2'] == correct_name)
    ]

    corrections = {}
    for _, row in harry_potter_similarities.iterrows():
        incorrect_name = row['Value 2'] if row['Value 1'] == correct_name else row['Value 1']
        corrections[incorrect_name] = correct_name

    return corrections



### 6. 🔍 Validate and Fix Data Types

In [10]:
def check_formats(df: pd.DataFrame, expected_formats: dict):
    incorrect_formats = []
    for column, datatype in df.dtypes.to_dict().items():
        expected_type = expected_formats.get(column)
        if datatype != expected_type:
            incorrect_formats.append((column, datatype, expected_type))
    
    incorrect_columns = [column[0] for column in incorrect_formats]
    correct_format_count = len([column for column in df.columns if column not in incorrect_columns])
    print(f'Correct Format Count: {correct_format_count} \nCorrect Columns: {[column for column in df.columns if column not in incorrect_columns]}')
    if incorrect_formats:
        print("Incorrect formats found! Format Validation Failed ❌")
        print("Incorrect Formats:")
        return pd.DataFrame(incorrect_formats, columns=['Column Name', 'Current Format', 'Expected Format'])
    else:
        print("All formats are correct! Format Validation Successful ✅")

In [11]:
def convert_column_types(df: pd.DataFrame, col_types: dict):
    try:
        for column, datatype in col_types.items():
            df[column] = df[column].astype(datatype)
        return df
    except Exception as e:
        print(f"Error converting column types: {e}")
        return df

###  🚀 Apply All Validations and Cleaning Steps

### 🧹 1. Remove All Nulls

In [12]:
print(df.isnull().sum())

Student Name                           13440
House                                   4000
Year                                       0
Wand Type                                  0
Pet                                     7946
Potions Grade                           3440
Defense Against the Dark Arts Grade        0
Transfiguration Grade                      0
Spells Learned                             0
Quidditch Position                      9504
Points Earned for House                    0
Detentions                                 0
Participation in Dueling Club              0
Triwizard Tournament Involvement        5920
Items Owned                                0
Knuts Spent in Hogsmeade                   0
Attendance at Classes                      0
Magical Accidents                          0
fileName                                   0
loadDateTimeStamp                          0
dtype: int64


In [13]:
df = fill_nulls_with_most_frequent(df)
df.isnull().sum()

Student Name                           0
House                                  0
Year                                   0
Wand Type                              0
Pet                                    0
Potions Grade                          0
Defense Against the Dark Arts Grade    0
Transfiguration Grade                  0
Spells Learned                         0
Quidditch Position                     0
Points Earned for House                0
Detentions                             0
Participation in Dueling Club          0
Triwizard Tournament Involvement       0
Items Owned                            0
Knuts Spent in Hogsmeade               0
Attendance at Classes                  0
Magical Accidents                      0
fileName                               0
loadDateTimeStamp                      0
dtype: int64

### ✨ 2. Clean Punctuation in `Quidditch Position`

In [14]:
df["Quidditch Position"].unique()

array(['Beater', 'Chaser', 'Seeker', 'Keeper', 'See;ker', 'Ch[aser',
       'K=eeper', 'C-haser', 'B_eater', 'Chase$r', 'K,eeper', 'B%eater',
       'Beat&er', 'K-eeper', 'Keep)er', 'Beate#r', 'Keep.er', 'Seek#er',
       'Ke^eper', 'S?eeker', 'Keepe_r', 'Keeper*', 'Keeper>', 'Keepe+r',
       'Ke-eper', 'Beater(', 'Kee[per', 'Ch`aser', 'C)haser', 'Ke@eper',
       'Seek=er', 'Beat\\er', 'Chase^r', "S'eeker", 'Beate.r', 'Keeper\\',
       'Keeper=', 'Se[eker', 'Ke`eper', "C'haser", 'Beate%r', 'Kee=per',
       'Chase(r', 'Bea-ter', 'Cha[ser', 'Se(eker', 'B{eater', 'Beate,r',
       'Keep|er', 'Keep"er', 'Keeper~', 'Seeker*', 'Seek(er', 'Beate<r',
       'See[ker', 'Beate\\r', 'Keeper,', 'Seeker;', 'Keeper[', 'Be]ater',
       'S[eeker', 'Kee*per', "Keeper'", 'Kee]per', 'Ch!aser', 'Chas@er',
       'Kee\\per', 'Seeke)r', 'K^eeper', 'Ch,aser', 'Chaser|', 'Chas,er',
       "B'eater", 'Beater^', 'Keepe#r', 'Keep}er', 'Ch?aser', 'Beate[r',
       'Kee?per', 'B-eater', 'K\\eeper', 'Cha)ser',

In [15]:
df = clean_quidditch_position(df)
df["Quidditch Position"].unique()

array(['Beater', 'Chaser', 'Seeker', 'Keeper'], dtype=object)

### 🔎 3. Find Misspellings in `Student Name`

In [16]:
threshold = 0.8
misspellings_df = check_misspelling(df, column="Student Name", threshold=threshold)

misspellings_df

,Value 1,Value 2,Similarity
0,Parvati MalfoWy,Parvati Malfoy,0.9231
1,Parvati MalfoWy,Parvati Malfovy,0.9231
2,Parvati MalfoWy,Parvati Maltfoy,0.9231
3,Parvati MalfoWy,Parvati Malqfoy,0.8571
4,Parvati MalfoWy,Parvati Marlfoy,0.9231
...,...,...,...
24768,Hermione aFinnigan,Hermione Fiannigan,1.0000
24769,Susan Spinneot,Seamus Spinnaet,0.8333
24770,Hermieone Finnigan,Hermione Fiannigan,1.0000
24771,Seamus Spinnaet,Seamus SpinFnet,0.9167


### 🏠 4. Clean `House` Names

In [17]:
df["House"].unique()

array(['Slytherin', 'Gryffindor', 'Ravenclaw', 'Hufflepuff', 'SLYTHERIN',
       'RAVENCLAW', 'HUFFLEPUFF', 'GRYFFINDOR'], dtype=object)

In [18]:
df = clean_House(df)
df['House'].unique()

array(['Slytherin', 'Gryffindor', 'Ravenclaw', 'Hufflepuff'], dtype=object)

### ✍️ 5. Correct `Harry Potter` Name Variants

In [20]:
correct_name = 'Harry Potter'
corrections = generate_corrections_mapping(misspellings_df, correct_name)

print("The corrections are:")
print(corrections) 

The corrections are:
{'Harry cPotter': 'Harry Potter', 'Harry Pottefr': 'Harry Potter', 'Harry Pojtter': 'Harry Potter', 'Harry Poktter': 'Harry Potter', 'Harry PotterO': 'Harry Potter', 'Haqrry Potter': 'Harry Potter', 'Harry PoJtter': 'Harry Potter', 'HarrLy Potter': 'Harry Potter', 'Harry PotNter': 'Harry Potter'}


In [21]:
df['Student Name'] = df['Student Name'].replace(corrections)

### 🛠️ 6. Validate and Fix Data Types 

In [22]:
expected_formats = {
    'Student Name': 'string',
    'House': 'string',
    'Year': 'int32',
    'Wand Type': 'string',
    'Pet': 'string',
    'Potions Grade': 'string',
    'Defense Against the Dark Arts Grade': 'string',
    'Transfiguration Grade': 'string',
    'Spells Learned': 'int32',
    'Quidditch Position': 'string',
    'Points Earned for House': 'int32',
    'Detentions': 'int32',
    'Participation in Dueling Club': 'string',
    'Triwizard Tournament Involvement': 'string',
    'Items Owned': 'string',
    'Knuts Spent in Hogsmeade': 'int32',
    'Attendance at Classes': 'int32',
    'Magical Accidents': 'int32',
    'fileName': 'string',
    'loadDateTimeStamp': 'datetime64[ns]'
}

check_formats(df, expected_formats)

Correct Format Count: 1 
Correct Columns: ['loadDateTimeStamp']
Incorrect formats found! Format Validation Failed ❌
Incorrect Formats:


,Column Name,Current Format,Expected Format
0,Student Name,object,string
1,House,object,string
2,Year,int64,int32
3,Wand Type,object,string
4,Pet,object,string
5,Potions Grade,object,string
6,Defense Against the Dark Arts Grade,object,string
7,Transfiguration Grade,object,string
8,Spells Learned,int64,int32
9,Quidditch Position,object,string


In [23]:
convert_column_types(df, expected_formats)

,Student Name,House,Year,Wand Type,Pet,Potions Grade,Defense Against the Dark Arts Grade,Transfiguration Grade,Spells Learned,Quidditch Position,Points Earned for House,Detentions,Participation in Dueling Club,Triwizard Tournament Involvement,Items Owned,Knuts Spent in Hogsmeade,Attendance at Classes,Magical Accidents,fileName,loadDateTimeStamp
0,Seamus Potter,Slytherin,2,"9 inches, elm, veela hair core",Dragon,Outstanding,Exceeds Expectations,Outstanding,26,Beater,-100,3,No,No,"Time-Turner, Marauder's Map",88,74,5,Hogwarts_Student_Data_20240101.csv,2024-12-05 22:53:29.684926
1,Ginny Spinnet,Gryffindor,6,"9 inches, elm, veela hair core",Cat,Exceeds Expectations,Acceptable,Outstanding,11,Chaser,139,2,No,No,"Firebolt, Sneakoscope, Extendable Ears",99,67,6,Hogwarts_Student_Data_20240101.csv,2024-12-05 22:53:29.684926
2,Padma Parkinson,Gryffindor,4,"11 inches, holly, phoenix feather core",Muggle Born,Outstanding,Acceptable,Outstanding,19,Seeker,93,2,No,No,Invisibility Cloak,342,88,6,Hogwarts_Student_Data_20240101.csv,2024-12-05 22:53:29.684926
3,Dean Malfoy,Ravenclaw,1,"12 inches, yew, unicorn hair core",Toad,Exceeds Expectations,Exceeds Expectations,Exceeds Expectations,16,Seeker,-73,2,No,No,Time-Turner,233,45,5,Hogwarts_Student_Data_20240101.csv,2024-12-05 22:53:29.684926
4,Dean Malfoy,Ravenclaw,1,"10 inches, oak, dragon heartstring core",Owl,Exceeds Expectations,Outstanding,Exceeds Expectations,9,Keeper,90,0,No,Yes,"Marauder's Map, Sneakoscope",473,27,7,Hogwarts_Student_Data_20240101.csv,2024-12-05 22:53:29.684926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,Susan Potter,Slytherin,1,"11 inches, holly, phoenix feather core",Cat,Outstanding,Outstanding,Exceeds Expectations,24,Seeker,90,5,Yes,Yes,"Marauder's Map, Firebolt",399,40,1,Hogwarts_Student_Data_20240601.csv,2024-12-05 22:53:29.754107
47996,Alicyia Boot,Gryffindor,4,"11 inches, holly, phoenix feather core",Toad,Exceeds Expectations,Poor,Poor,28,Seeker,-9,1,Yes,Yes,"Extendable Ears, Marauder's Map, Invisibility ...",299,27,5,Hogwarts_Student_Data_20240601.csv,2024-12-05 22:53:29.754107
47997,Terry Granger,Ravenclaw,7,"9 inches, elm, veela hair core",Owl,Outstanding,Poor,Exceeds Expectations,12,Seeker,-47,4,Yes,No,"Sneakoscope, Firebolt",124,27,10,Hogwarts_Student_Data_20240601.csv,2024-12-05 22:53:29.754107
47998,Ron Bell,Hufflepuff,1,"9 inches, elm, veela hair core",Dragon,Exceeds Expectations,Acceptable,Acceptable,11,Chaser,75,5,Yes,No,"Firebolt, Sneakoscope",362,49,7,Hogwarts_Student_Data_20240601.csv,2024-12-05 22:53:29.754107


In [24]:
check_formats(df, expected_formats)

Correct Format Count: 20 
Correct Columns: ['Student Name', 'House', 'Year', 'Wand Type', 'Pet', 'Potions Grade', 'Defense Against the Dark Arts Grade', 'Transfiguration Grade', 'Spells Learned', 'Quidditch Position', 'Points Earned for House', 'Detentions', 'Participation in Dueling Club', 'Triwizard Tournament Involvement', 'Items Owned', 'Knuts Spent in Hogsmeade', 'Attendance at Classes', 'Magical Accidents', 'fileName', 'loadDateTimeStamp']
All formats are correct! Format Validation Successful ✅


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48000 entries, 0 to 47999
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Student Name                         48000 non-null  string        
 1   House                                48000 non-null  string        
 2   Year                                 48000 non-null  int32         
 3   Wand Type                            48000 non-null  string        
 4   Pet                                  48000 non-null  string        
 5   Potions Grade                        48000 non-null  string        
 6   Defense Against the Dark Arts Grade  48000 non-null  string        
 7   Transfiguration Grade                48000 non-null  string        
 8   Spells Learned                       48000 non-null  int32         
 9   Quidditch Position                   48000 non-null  string        
 10  Points Ear

### 💾 Save the Silver Layer as `Silver.parquet`

In [26]:
df.to_parquet('./Medallion Architecture/Silver/Silver.parquet', index=False)